# 1년치 애플 주식 데이터 수집 코드

In [1]:
#!/usr/bin/env python3
import yfinance as yf
import pandas as pd
import datetime


def fetch_news(ticker_symbol: str) -> pd.DataFrame:
    """
    yfinance를 이용해 지정한 티커의 최신 뉴스 헤드라인을 가져와
    title, summary, link, publisher, pubDate 컬럼만 리턴합니다.
    """
    ticker = yf.Ticker(ticker_symbol)
    raw = ticker.news

    # raw가 {'content': [...]} 형태면 content 리스트를 꺼내고, 아니라면 raw 자체 사용
    articles = raw.get('content') if isinstance(raw, dict) and 'content' in raw else raw

    # 딕셔너리 리스트를 평탄화하여 DataFrame 생성
    df = pd.json_normalize(articles)

    # 필요한 컬럼만 골라서 정리
    df_clean = pd.DataFrame({
        'title':     df.get('content.title'),
        'summary':   df.get('content.summary'),
        'link':      df.get('content.clickThroughUrl.url').fillna(df.get('content.canonicalUrl.url')),
        'publisher': df.get('content.provider.displayName'),
        'pubDate':   df.get('content.pubDate')
    })
    return df_clean


def fetch_stock_data(ticker_symbol: str,
                     period: str = '1y',
                     interval: str = '1d') -> pd.DataFrame:
    """
    yfinance의 download 기능을 이용해 지정한 티커의 주식 데이터를 가져옵니다.
    - period: '5d', '1mo', '3mo', '1y' 등
    - interval: '1d', '1wk', '1mo', '1m' 등
    반환 컬럼: Date, Open, High, Low, Close, Adj_Close, Volume
    """
    # auto_adjust=False로 Adj Close 컬럼 포함
    df = yf.download(
        tickers=ticker_symbol,
        period=period,
        interval=interval,
        auto_adjust=False,
        progress=False
    )
    if df.empty:
        return pd.DataFrame()

    df = df.reset_index()
    df = df.rename(columns={'Adj Close': 'Adj_Close'})
    return df[['Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume']]


if __name__ == "__main__":
    ticker = "AAPL"  # 예시로 Apple Inc. 티커 사용

    # 2) 주식 데이터 추출
    df_stock = fetch_stock_data(ticker, period='1y', interval='1d')
    print(f"=== {ticker} Stock Prices (Last {len(df_stock)} days) ===")
    if not df_stock.empty:
        print(df_stock.to_string(index=False))
    else:
        print("주식 데이터를 가져올 수 없습니다.")

    # 3) CSV 파일로 저장
    today = datetime.date.today().isoformat()
    stock_filename = f"{ticker}_stock_{today}.csv"
    df_stock.to_csv(stock_filename, index=False)
    print(f"✅ 주식 데이터가 '{stock_filename}' 파일로 저장되었습니다.")

=== AAPL Stock Prices (Last 250 days) ===
      Date       Open       High        Low      Close  Adj_Close    Volume
                 AAPL       AAPL       AAPL       AAPL       AAPL      AAPL
2024-05-24 188.820007 190.580002 188.039993 189.979996 189.095657  36294600
2024-05-28 191.509995 193.000000 189.100006 189.990005 189.105606  52280100
2024-05-29 189.610001 192.250000 189.509995 190.289993 189.404221  53068000
2024-05-30 190.759995 192.179993 190.630005 191.289993 190.399567  49947900
2024-05-31 191.440002 192.570007 189.910004 192.250000 191.355103  75158300
2024-06-03 192.899994 194.990005 192.520004 194.029999 193.126816  50080500
2024-06-04 194.639999 195.320007 193.029999 194.350006 193.445312  47471400
2024-06-05 195.399994 196.899994 194.869995 195.869995 194.958237  54156800
2024-06-06 195.690002 196.500000 194.169998 194.479996 193.574707  41181800
2024-06-07 194.649994 196.940002 194.139999 196.889999 195.973495  53103900
2024-06-10 196.899994 197.300003 192.149994 19